<a href="https://colab.research.google.com/github/SeongGwan-a/Project-Classification-of-novelists/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/소설작가분류/train.csv")
test = pd.read_csv("/content/drive/MyDrive/소설작가분류/test_x.csv")

In [ ]:
text = train["text"]

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
text[1]

'“Your sister asked for it, I suppose?”'

In [ ]:
word_tokenize(text[0])

['He',
 'was',
 'almost',
 'choking',
 '.',
 'There',
 'was',
 'so',
 'much',
 ',',
 'so',
 'much',
 'he',
 'wanted',
 'to',
 'say',
 ',',
 'but',
 'strange',
 'exclamations',
 'were',
 'all',
 'that',
 'came',
 'from',
 'his',
 'lips',
 '.',
 'The',
 'Pole',
 'gazed',
 'fixedly',
 'at',
 'him',
 ',',
 'at',
 'the',
 'bundle',
 'of',
 'notes',
 'in',
 'his',
 'hand',
 ';',
 'looked',
 'at',
 'odin',
 ',',
 'and',
 'was',
 'in',
 'evident',
 'perplexity',
 '.']

# **A**

In [ ]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 
warnings.filterwarnings(action='ignore')

In [ ]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [ ]:
target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [ ]:
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)


def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)


stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [ ]:
train['text'] = train['text'].str.lower().apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].str.lower().apply(alpha_num).apply(remove_stopwords)

In [ ]:
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values
print(X_train.shape, X_test.shape, y.shape)

(54879,) (19617,) (54879,)


In [ ]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly not occurred said yes like'],
      dtype=object)

In [ ]:
vocab_size = 20000
embedding_dim = 64
max_length = 500
padding_type='post'

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
print(trn.shape, tst.shape)

(54879, 500) (19617, 500)


In [ ]:
train_sequences

[[141,
  7259,
  20,
  20,
  316,
  21,
  229,
  3068,
  36,
  452,
  2735,
  1202,
  5024,
  1701,
  1207,
  45,
  57,
  1,
  1214,
  2828],
 [217, 58, 221],
 [682,
  5,
  59,
  250,
  13200,
  4744,
  38,
  194,
  4540,
  4745,
  1232,
  485,
  2,
  625,
  499,
  699,
  501,
  6,
  1,
  79,
  107,
  1,
  671,
  862,
  50,
  194,
  424,
  5323,
  289,
  3],
 [251,
  5025,
  874,
  1237,
  43,
  8442,
  830,
  1504,
  121,
  254,
  24,
  1147,
  617,
  5324,
  502,
  1,
  66,
  1735,
  135,
  399,
  2166,
  1789,
  2122,
  1626,
  617,
  106,
  3758,
  6419,
  1901,
  137,
  1790],
 [1381,
  346,
  1,
  1555,
  106,
  31,
  488,
  1478,
  975,
  211,
  2038,
  104,
  6666,
  4012,
  891,
  16700,
  6979,
  9554,
  83,
  200],
 [12, 3420, 3, 3177, 7, 2, 1238, 24, 717],
 [2, 662, 431, 1556, 394, 7, 244, 264, 6420, 309, 76, 13],
 [1096,
  602,
  10,
  3,
  14,
  944,
  10,
  4,
  6157,
  74,
  3354,
  1768,
  10,
  15,
  39,
  658,
  1003,
  703,
  8,
  1505,
  303,
  5162,
  11998,
  698

In [ ]:
trn

array([[  141,  7259,    20, ...,     0,     0,     0],
       [  217,    58,   221, ...,     0,     0,     0],
       [  682,     5,    59, ...,     0,     0,     0],
       ...,
       [ 2407, 19738,   126, ...,     0,     0,     0],
       [  316,  3532,   164, ...,     0,     0,     0],
       [  203,     2,   811, ...,     0,     0,     0]], dtype=int32)

In [ ]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [ ]:
def get_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(64)),
        Dense(n_class, activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=.01))
    return model

In [ ]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = get_model()
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict(tst) / n_fold

training model for CV #1
Epoch 1/10
86/86 [==============================] - 1269s 15s/step - loss: 1.0386 - val_loss: 0.7825
Epoch 2/10
86/86 [==============================] - 1141s 13s/step - loss: 0.5684 - val_loss: 0.7318
Epoch 3/10
86/86 [==============================] - 863s 10s/step - loss: 0.4121 - val_loss: 0.7974
Epoch 4/10
86/86 [==============================] - 956s 11s/step - loss: 0.3312 - val_loss: 0.9049
Epoch 5/10
86/86 [==============================] - 936s 11s/step - loss: 0.2735 - val_loss: 0.9873
Epoch 00005: early stopping
training model for CV #2
Epoch 1/10
86/86 [==============================] - 961s 11s/step - loss: 1.0452 - val_loss: 0.7890
Epoch 2/10
86/86 [==============================] - 909s 11s/step - loss: 0.5604 - val_loss: 0.7327
Epoch 3/10
86/86 [==============================] - 943s 11s/step - loss: 0.4070 - val_loss: 0.8188
Epoch 4/10
86/86 [==============================] - 919s 11s/step - loss: 0.3253 - val_loss: 0.8677
Epoch 5/10
86/86 [==

In [ ]:
print(clf.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 64)           1280000   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 500, 128)          66048     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 645       
Total params: 1,445,509
Trainable params: 1,445,509
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
baseline = pd.read_csv("/content/drive/MyDrive/소설작가분류/sample_submission.csv", index_col=0)
print(baseline.shape)
baseline.head()

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [ ]:
baseline[baseline.columns] = p_tst
baseline.head()

,0,1,2,3,4
index,,,,,
0,0.0427,0.5322,0.3575,0.0612,0.0065
1,0.1345,0.3892,0.0160,0.0596,0.4006
2,0.9670,0.0232,0.0020,0.0035,0.0043
3,0.0120,0.0140,0.8614,0.0151,0.0975
4,0.3241,0.0855,0.0350,0.4885,0.0669


In [ ]:
baseline.to_csv("baseline_file")